# Testing a "Data Availabilty Statement"

This is a quick test of being able to get the data used in the article J. Birch and J. Hart. (2017). Social Networks and Northern Iroquoian Confederacy Dynamics, _American Antiquity 83_(1): 13-33. &lt;https://doi.org/10.1017/aaq.2017.59&gt;

Assuming the code runs, an interactive network visualization will be created is that is a rough equivalent of figure 5(a) in the text.

In [9]:
import folium
from io import BytesIO
import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns
import subprocess

%matplotlib inline

In [8]:
# data from 

load_from_url = True # set to False to read from local copy of file
if load_from_url:
    cmd = ['curl',
       'https://static.cambridge.org/resource/id/urn:cambridge.org:id:binary:20180119015354028-0906:S0002731617000592:S0002731617000592sup001.xls']
    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    o, e = proc.communicate()

    from_xls_df = pd.read_excel(BytesIO(o))
    print("loaded from url")
else:
    from_xls_df = pd.read_excel("local_copy_of_data.xls")

loaded from url


In [6]:
iroquois_df = from_xls_df.iloc[0:77,np.r_[2, 3:32]] # this could also be done with a query that used  values
iroquois_df.to_csv("BR.csv", index=False)

In [10]:
# ONLY RUN THIS CELL IF YOUR ENVIRONMENT HAS r2py INSTALLED.
# WON'T WORK IN MYBINDER
# THE OUTPUT IS CACHED SO YOU CAN SKIP.

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

from rpy2.robjects.conversion import localconverter

dists_r = ro.r('''# Script by Matt Peeples http://www.mattpeeples.net/br.html
library(statnet) # initialize necessary library

# Function for calculating Brainerd-Robinson (BR) coefficients
BR <- function(x) {
rd <- dim(x)[1]
results <- matrix(0,rd,rd)
for (s1 in 1:rd) {
for (s2 in 1:rd) {
x1Temp <- as.numeric(x[s1, ])
x2Temp <- as.numeric(x[s2, ])
br.temp <- 0
results[s1,s2] <- 200 - (sum(abs(x1Temp - x2Temp)))}}
row.names(results) <- row.names(x)
colnames(results) <- row.names(x)
return(results)}

# Obtain input table 
br.tab1 <- read.table(file='BR.csv', sep=',', header=T, row.names=1) # the name of each row (site name) should be the first column in the input table
br.tab1 <- na.omit(br.tab1)

per <- 2

# If user selects counts, convert data to percents and run simulation
if (per == 2) {
br.tab <- prop.table(as.matrix(br.tab1),1)*100
br.dat <- BR(br.tab) # actual BR values

# Calculate the proportions of each category in the original data table
c.sum <- matrix(0,1,ncol(br.tab1))
for (i in 1:ncol(br.tab1)) {
c.temp <- sum(br.tab1[,i])
c.sum[,i] <- c.temp}
p.grp <- prop.table(c.sum)

# Create random sample of a specified sample size
MC <- function(x,s.size) {
v3 <- matrix(0,ncol(x),1)
rand.tab.2 <- matrix(0,ncol(x),1)
v <- sample(ncol(x),s.size,prob=p.grp,replace=T)
v2 <- as.matrix(table(v))
for (j in 1:nrow(v2)) {
v3.temp <- v2[j,]
v3[j,1] <- v3.temp}
rand.tab <- as.matrix(prop.table(v3))*100
rand.tab.2[,1] <- rand.tab
return(rand.tab.2)}

r.sums <- as.matrix(rowSums(br.tab1)) # Calculate sample size by row

# Initate random samples for all rows
BR_rand <- function(x) {
rand.test <- matrix(0,ncol(x),nrow(r.sums))
for (i in 1:nrow(x)) {
rand.test[,i] <- MC(br.tab1,r.sums[i,])}
return(rand.test)}

br.diff.out <- matrix(0,nrow(br.dat),ncol(br.dat)) # Initialize table

randruns <- 100

# Run MC simulation of BR values
for (i in 1:randruns) {
br.temp <- BR_rand(br.tab1)
br.temp <- t(br.temp)
br.test <- BR(br.temp)
br.diff <- br.dat - br.test
br.diff2 <- event2dichot(br.diff,method='absolute',thresh=0)
br.diff.out <- br.diff.out + br.diff2}

br.diff.out <- br.diff.out / randruns # Calculate probabilities based on random runs
row.names(br.diff.out) <- row.names(br.tab1)
colnames(br.diff.out) <- row.names(br.tab1)

write.table(br.diff.out,file='BR_prob.csv',sep=',') # Write output

} # close if statement for count data

# Recalculate actual BR values and output to file
br.tab <- prop.table(as.matrix(br.tab1),1)*100
br.dat <- BR(br.tab)
write.table(br.dat,file='BR_out.csv',sep=',') # Write output

 

# end of script

''')

R[write to console]: Loading required package: tergm

R[write to console]: Loading required package: ergm

R[write to console]: Loading required package: network

R[write to console]: network: Classes for Relational Data
Version 1.13.0.1 created on 2015-08-31.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.


R[write to console]: 
ergm: version 3.9.4, created on 2018-08-15
Copyright (c) 2018, Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Carter T. Butts, University of California -- Irvine
                    Steven M. Go

In [ ]:
ir_matrix_df = pd.read_csv("BR_out.csv")
ir_matrix_df

In [ ]:
G_iroquois = nx.from_pandas_adjacency(ir_matrix_df)

In [ ]:
e_dict = dict(G_iroquois.edges)

for e in e_dict:
    if e[0] == e[1]:
        G_iroquois.remove_edge(e[0],e[1])

In [ ]:
e_dict = dict(G_iroquois.edges)

for e in e_dict:
    # G_iroquois.edges[('Burke','Frank')]
    w = G_iroquois.edges[(e[0],e[1])]['weight']
    if w <= 110:
        G_iroquois.remove_edge(e[0],e[1])


In [ ]:
# commented out because it doesn't look that nice. 
# could be fixed but interactive network the real goal

# nx.draw(G_iroquois, nx.spring_layout(G_iroquois, iterations=10), node_size=5, k = .0001)

In [ ]:
map_ir = folium.Map( crs = 'Simple', tiles = None, zoom_start = 5)

pos = nx.spring_layout(G_iroquois, iterations=500, k = .15)

f_scale = 5000

for s,t in G_iroquois.edges:
    s_loc = tuple([pos[s][1]*f_scale, pos[s][0]*f_scale])
    t_loc = tuple([pos[t][1]*f_scale, pos[t][0]*f_scale])
    # print(s_loc)
    folium.PolyLine([s_loc,t_loc], color = 'black', weight = .2, opacity = .5).add_to(map_ir)

groups = {r[0]:r[1] for i,r in from_xls_df[['Site','Group']].iterrows()}

groups_u = from_xls_df.Group.dropna().unique()
palette = sns.color_palette(None, len(groups_u)).as_hex()
colors = {groups_u[i]:palette[i] for i in range(len(groups_u)) }

for p in pos:
    folium.CircleMarker([pos[p][1]*f_scale,pos[p][0]*f_scale],
                        popup = "{} ({})".format(p,groups[p]),
                        radius = 4,
                        color = colors[groups[p]]).add_to(map_ir)

map_ir.save("iroquois.html")    

map_ir